In [109]:
import pandas as pd
import requests
from fredapi import Fred

ModuleNotFoundError: No module named 'fredapi'

In [93]:
api_key = 'AJLfYJhBQg4BBHCEgGquNBoCeDFWaJK8R1LdyKrc'
url = "https://api.eia.gov/v2/co2-emissions/co2-emissions-aggregates/data"
params = {
    "api_key": api_key,
    "frequency": "annual",
    "data[0]": ["value"],
    "facets": {},
    "start": "1990",
    "end": "1995",
    "sort": [],
    "offset": 0,
    "length": 5000
}

data_all = []

# Maximum length of EIA API output is 5000 rows; while loop below gets all rows

while True:
    response = requests.get(url, params=params)     
    data = response.json()        
    data_all.extend(data["response"]["data"])    
    if len(data["response"]["data"]) < 5000:
        break
    
    params['offset'] += 5000

states = pd.DataFrame(data_all)

In [94]:
states

,period,sectorId,sector-name,fuelId,fuel-name,stateId,state-name,value,value-units
0,1990,RC,Residential carbon dioxide emissions,CO,Coal,OK,Oklahoma,.000048,million metric tons of CO2
1,1990,RC,Residential carbon dioxide emissions,PE,Petroleum,OK,Oklahoma,.309053,million metric tons of CO2
2,1990,RC,Residential carbon dioxide emissions,NG,Natural Gas,OK,Oklahoma,3.552208,million metric tons of CO2
3,1990,RC,Residential carbon dioxide emissions,TO,All Fuels,OK,Oklahoma,3.861309,million metric tons of CO2
4,1990,CC,Commercial carbon dioxide emissions,CO,Coal,OK,Oklahoma,.000191,million metric tons of CO2
...,...,...,...,...,...,...,...,...,...
7483,1995,EC,Electric Power carbon dioxide emissions,TO,All Fuels,WY,Wyoming,39.824829,million metric tons of CO2
7484,1995,TT,Total carbon dioxide emissions from all sectors,CO,Coal,WY,Wyoming,44.035932,million metric tons of CO2
7485,1995,TT,Total carbon dioxide emissions from all sectors,PE,Petroleum,WY,Wyoming,8.803001,million metric tons of CO2
7486,1995,TT,Total carbon dioxide emissions from all sectors,NG,Natural Gas,WY,Wyoming,5.339128,million metric tons of CO2


In [95]:
states['sector-name'].value_counts()

sector-name
Residential carbon dioxide emissions               1248
Commercial carbon dioxide emissions                1248
Industrial carbon dioxide emissions                1248
Transportation carbon dioxide emissions            1248
Electric Power carbon dioxide emissions            1248
Total carbon dioxide emissions from all sectors    1248
Name: count, dtype: int64

In [100]:
# Get emissions from electric power production
states_total = states[(states['sector-name'] == 'Total carbon dioxide emissions from all sectors') & (states['fuelId'] == 'TO')]
states_total

,period,sectorId,sector-name,fuelId,fuel-name,stateId,state-name,value,value-units
23,1990,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,OK,Oklahoma,87.852162,million metric tons of CO2
47,1991,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,OK,Oklahoma,89.171638,million metric tons of CO2
71,1992,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,OK,Oklahoma,92.241151,million metric tons of CO2
95,1993,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,OK,Oklahoma,95.2188,million metric tons of CO2
118,1994,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,OK,Oklahoma,93.41324,million metric tons of CO2
...,...,...,...,...,...,...,...,...,...
7411,1995,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,US,United States,5297.044254,million metric tons of CO2
7415,1992,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,WY,Wyoming,61.19218,million metric tons of CO2
7439,1993,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,WY,Wyoming,58.197343,million metric tons of CO2
7463,1994,TT,Total carbon dioxide emissions from all sectors,TO,All Fuels,WY,Wyoming,60.385852,million metric tons of CO2


In [101]:
# drop unnecessary columns
states_total = states_total[['period','stateId','value']]

In [102]:
# rename columns
states_total = states_total.rename(columns={'period': 'date', 'stateId': 'state', 'value': 'total_co2'})

In [103]:
states_total

,date,state,total_co2
23,1990,OK,87.852162
47,1991,OK,89.171638
71,1992,OK,92.241151
95,1993,OK,95.2188
118,1994,OK,93.41324
...,...,...,...
7411,1995,US,5297.044254
7415,1992,WY,61.19218
7439,1993,WY,58.197343
7463,1994,WY,60.385852


In [104]:
# group by year and state, sum total_co2
states_total = states_total.groupby(['date', 'state']).total_co2.sum().reset_index()

In [106]:
states_total

,date,state,total_co2
0,1990,AK,33.957883
1,1990,AL,109.751599
2,1990,AR,50.827554
3,1990,AZ,63.093644
4,1990,CA,360.335591
...,...,...,...
307,1995,VT,5.990927
308,1995,WA,78.043766
309,1995,WI,96.603289
310,1995,WV,104.870887


In [108]:
fred = Fred(api_key='14b0cdcf3e1553c5aa4d294486de3fb7')
states = [state for states in states_total['state'].unique()]
unemployment_data = {}

NameError: name 'Fred' is not defined